In [3]:
import pandas as pd
import numpy as np
import sqlite3 as db
from datetime import datetime
import urllib
import os
from pyquery import PyQuery  


con = db.connect(database="../db.sqlite")

In [36]:
cursor = con.cursor()
cursor.execute("select CountryCode, Name from Country")

list = []

for row in cursor.fetchall():
    list.append({
        "CountryCode": row[0],
        "Country": row[1]
    })
    

def getCountryCode(name):
    
    if name == "United Kingdom":
        return "GBR"
    elif name == "Russian Federation":
        return "RUS"
    elif name == "Korea":
        return "KOR"
    elif name == "Slovak Republic":
        return "SVK"
    
    for c in list:
        if c["Country"] == name:
            return c["CountryCode"]
        
    return None

In [4]:

pq = PyQuery(url="https://en.wikipedia.org/wiki/List_of_countries_by_tax_rates")

In [54]:
countries = pq(".wikitable tbody tr td:nth-child(1)")

minWages = pq(".wikitable tbody tr td:nth-child(3)")



data = []


for i in range(0, countries.length):
    countryName = PyQuery(countries[i])("a")[0].text
    
    countryCode =  getCountryCode(countryName)
    
    if countryCode is not None:
        taxTxt = PyQuery(minWages[i]).text()
        taxRate = taxTxt.split('%')[0]
        
        taxRate = taxRate.replace("from ", "")
        
        
        if taxRate == "N/A":
            taxRate = "0"
        
        
        data.append({
            "Country_id": countryCode,
            "Rate": float(taxRate) / 100,
            "Source": "wiki",
            "Profession": "Min Wage",
            "Date": datetime(2020,1,1),            
        })
        
    
    
df = pd.DataFrame(data)



In [55]:
df.to_sql("TaxWedge", con,if_exists="append", index=False)